In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchsummary import summary
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_trans = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])
val_trans = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

In [3]:
train_ds = ImageFolder(root='/Users/arponbiswas/Computer-Vision-Projects/Image_classification_projects/Ant_Bee_Classification/Data_samples', transform=train_trans)
val_ds = ImageFolder(root='/Users/arponbiswas/Computer-Vision-Projects/Image_classification_projects/Ant_Bee_Classification/Data_samples', transform=val_trans)

In [4]:
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=32, shuffle=False)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=5, padding=0)

        self.flatten = nn.Flatten()
        self.fc = nn.Linear(6*44*44, 1)

    def forward(self, x):
        x = f.relu(self.conv1(x))
        x = self.flatten(x)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
model = Model()
model = model.to(device)
if model.name:
    writer = SummaryWriter(f'/Users/arponbiswas/Computer-Vision-Projects/Image_classification_projects/Ant_Bee_Classification/TensorBoard-Graph/runs/{model.name}')

In [85]:
writer.add_graph(model, torch.rand(1, 3, 224, 224).to(device))

In [86]:
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 44, 44]             456
           Flatten-2                [-1, 11616]               0
            Linear-3                    [-1, 1]          11,617
Total params: 12,073
Trainable params: 12,073
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 0.18
Params size (MB): 0.05
Estimated Total Size (MB): 0.80
----------------------------------------------------------------


In [87]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [88]:
def train_val(epochs=10):
    print('Training and Validation Started')
    for epoch in tqdm(range(epochs), desc='Epoch Progress'):
        # Training
        model.train()
        train_losses = []
        val_losses = []
        train_preds = []
        train_targets = []
        val_preds = []
        val_targets = []
        for images, labels in train_dl:
            images, labels = images.to(device), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(images)
            preds = torch.round(outputs).squeeze(1)
            loss = loss_fn(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            train_preds.extend(preds.cpu().detach().numpy())
            train_targets.extend(labels.cpu().detach().numpy())
        # Validation
        model.eval()
        with torch.no_grad():
            for images, labels in val_dl:
                images, labels = images.to(device), labels.to(device).float()
                outputs = model(images)
                preds = torch.round(outputs).squeeze(1)
                loss = loss_fn(outputs, labels.unsqueeze(1))
                val_losses.append(loss.item())
                val_preds.extend(preds.cpu().detach().numpy())
                val_targets.extend(labels.cpu().detach().numpy())
        # Metrics
        train_acc = accuracy_score(train_targets, train_preds)
        val_acc = accuracy_score(val_targets, val_preds)
        train_loss = sum(train_losses)/len(train_losses)
        val_loss = sum(val_losses)/len(val_losses)
        train_f1 = f1_score(train_targets, train_preds)
        val_f1 = f1_score(val_targets, val_preds)
        train_recall = recall_score(train_targets, train_preds)
        val_recall = recall_score(val_targets, val_preds)
        train_precision = precision_score(train_targets, train_preds)
        val_precision = precision_score(val_targets, val_preds)
        train_cm = confusion_matrix(train_targets, train_preds)
        val_cm = confusion_matrix(val_targets, val_preds)
        # Tensorboard
        # Training
        fig_train, ax_train = plt.subplots(figsize=(6, 6))
        sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ant', 'Bee'], yticklabels=['Ant', 'Bee'])
        ax_train.set_xlabel('Predicted Labels')
        ax_train.set_ylabel('True Labels')
        ax_train.set_title('Train Confusion Matrix')
        writer.add_figure('Train/Confusion Matrix', fig_train, epoch)
        writer.add_scalar('Train/Loss', train_loss, epoch)
        writer.add_scalar('Train/Accuracy', train_acc, epoch)
        writer.add_scalar('Train/F1 Score', train_f1, epoch)
        writer.add_scalar('Train/Recall', train_recall, epoch)
        writer.add_scalar('Train/Precision', train_precision, epoch)
        plt.close(fig_train)
        # Validation
        fig_val, ax_val = plt.subplots(figsize=(6, 6))
        sns.heatmap(val_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ant', 'Bee'], yticklabels=['Ant', 'Bee'])
        ax_val.set_xlabel('Predicted Labels')
        ax_val.set_ylabel('True Labels')
        ax_val.set_title('Validation Confusion Matrix')
        writer.add_figure('Validation/Confusion Matrix', fig_val, epoch)
        writer.add_scalar('Validation/Loss', val_loss, epoch)
        writer.add_scalar('Validation/Accuracy', val_acc, epoch)
        writer.add_scalar('Validation/F1 Score', val_f1, epoch)
        writer.add_scalar('Validation/Recall', val_recall, epoch)
        writer.add_scalar('Validation/Precision', val_precision, epoch)
        plt.close(fig_val)
        writer.flush()
    writer.close()

In [89]:
train_val(epochs=10)

Training and Validation Started


Epoch Progress: 100%|██████████| 10/10 [00:01<00:00,  5.48it/s]


# This code is for training all models in one time

In [ ]:
def train_val(model, epochs=10):
    # Adding model to tensorboard
    model = model.to(device)
    if model.name:
        writer = SummaryWriter(f'/Users/arponbiswas/Computer-Vision-Projects/Image_classification_projects/Ant_Bee_Classification/TensorBoard-Graph/runs/{model.name}')
    writer.add_graph(model, torch.rand(1, 3, 224, 224).to(device))
    print(model.name)
    summary(model, input_size=(3, 224, 224))
    # Loss Function and Optimizer
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # Training and Validation
    print('Training and Validation Started')
    for epoch in tqdm(range(epochs), desc='Epoch Progress'):
        # Training
        model.train()
        train_losses = []
        val_losses = []
        train_preds = []
        train_targets = []
        val_preds = []
        val_targets = []
        for images, labels in train_dl:
            images, labels = images.to(device), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(images)
            preds = torch.round(outputs).squeeze(1)
            loss = loss_fn(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            train_preds.extend(preds.cpu().detach().numpy())
            train_targets.extend(labels.cpu().detach().numpy())
        # Validation
        model.eval()
        with torch.no_grad():
            for images, labels in val_dl:
                images, labels = images.to(device), labels.to(device).float()
                outputs = model(images)
                preds = torch.round(outputs).squeeze(1)
                loss = loss_fn(outputs, labels.unsqueeze(1))
                val_losses.append(loss.item())
                val_preds.extend(preds.cpu().detach().numpy())
                val_targets.extend(labels.cpu().detach().numpy())
        # Metrics
        train_acc = accuracy_score(train_targets, train_preds)
        val_acc = accuracy_score(val_targets, val_preds)
        train_loss = sum(train_losses)/len(train_losses)
        val_loss = sum(val_losses)/len(val_losses)
        train_f1 = f1_score(train_targets, train_preds)
        val_f1 = f1_score(val_targets, val_preds)
        train_recall = recall_score(train_targets, train_preds)
        val_recall = recall_score(val_targets, val_preds)
        train_precision = precision_score(train_targets, train_preds)
        val_precision = precision_score(val_targets, val_preds)
        train_cm = confusion_matrix(train_targets, train_preds)
        val_cm = confusion_matrix(val_targets, val_preds)
        # Adding metrics to tensorboard
        # Training
        fig_train, ax_train = plt.subplots(figsize=(6, 6))
        sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ant', 'Bee'], yticklabels=['Ant', 'Bee'])
        ax_train.set_xlabel('Predicted Labels')
        ax_train.set_ylabel('True Labels')
        ax_train.set_title('Train Confusion Matrix')
        writer.add_figure('Train/Confusion Matrix', fig_train, epoch)
        writer.add_scalar('Train/Loss', train_loss, epoch)
        writer.add_scalar('Train/Accuracy', train_acc, epoch)
        writer.add_scalar('Train/F1 Score', train_f1, epoch)
        writer.add_scalar('Train/Recall', train_recall, epoch)
        writer.add_scalar('Train/Precision', train_precision, epoch)
        plt.close(fig_train)
        # Validation
        fig_val, ax_val = plt.subplots(figsize=(6, 6))
        sns.heatmap(val_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ant', 'Bee'], yticklabels=['Ant', 'Bee'])
        ax_val.set_xlabel('Predicted Labels')
        ax_val.set_ylabel('True Labels')
        ax_val.set_title('Validation Confusion Matrix')
        writer.add_figure('Validation/Confusion Matrix', fig_val, epoch)
        writer.add_scalar('Validation/Loss', val_loss, epoch)
        writer.add_scalar('Validation/Accuracy', val_acc, epoch)
        writer.add_scalar('Validation/F1 Score', val_f1, epoch)
        writer.add_scalar('Validation/Recall', val_recall, epoch)
        writer.add_scalar('Validation/Precision', val_precision, epoch)
        plt.close(fig_val)
        writer.flush()
    writer.close()

In [ ]:
# for model in models:
#     train_val(model, epochs=10)